In [6]:
import cv2
import numpy as np
import tensorflow as tf

In [7]:
detections_global = []
selected_box = None
tracking = False
tracker = None

In [8]:
interpreter = tf.lite.Interpreter(model_path="detect.tflite")
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

height = input_details[0]['shape'][1]
width = input_details[0]['shape'][2]

In [9]:
labels = []
with open("labelmap.txt","r") as f:
    for line in f:
        labels.append(line.strip())

In [4]:
import cv2
import numpy as np
import tensorflow as tf

detections_global = []
selected_box = None
tracking = False
tracker = None
current_frame = None

interpreter = tf.lite.Interpreter(model_path="detect.tflite")
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

height = input_details[0]['shape'][1]
width  = input_details[0]['shape'][2]

labels = []
with open("labelmap.txt", "r") as f:
    for line in f:
        labels.append(line.strip())

def compute_iou(boxA, boxB):
    ax1, ay1, ax2, ay2 = boxA
    bx1, by1, bx2, by2 = boxB

    inter_x1 = max(ax1, bx1)
    inter_y1 = max(ay1, by1)
    inter_x2 = min(ax2, bx2)
    inter_y2 = min(ay2, by2)

    inter_area = max(0, inter_x2 - inter_x1) * max(0, inter_y2 - inter_y1)
    area_a = (ax2 - ax1) * (ay2 - ay1)
    area_b = (bx2 - bx1) * (by2 - by1)
    union = area_a + area_b - inter_area

    return inter_area / union if union > 0 else 0


def mouse_click(event, x, y, flags, param):
    global selected_box, tracking, tracker

    if event == cv2.EVENT_LBUTTONDOWN:
        for box in detections_global:
            x1, y1, x2, y2, label = box
            if x1 <= x <= x2 and y1 <= y <= y2:
                selected_box = (x1, y1, x2, y2)
                tracking = True
                tracker = cv2.legacy.TrackerKCF_create()
                tracker.init(current_frame, (x1, y1, x2 - x1, y2 - y1))
                print(f"Now tracking: {label}")
                break

def on_trackbar(val):
    pass


cap = cv2.VideoCapture(0)
cv2.namedWindow("Object Detection")
cv2.setMouseCallback("Object Detection", mouse_click)

while True:
    ret, frame = cap.read()
    if not ret:                      
        break

    current_frame = frame.copy()
    detections_global = []           

    h, w, _ = frame.shape

    img = cv2.resize(frame, (width, height))
    input_data = np.expand_dims(img, axis=0).astype(np.uint8)

    interpreter.set_tensor(input_details[0]['index'], input_data)
    interpreter.invoke()

    boxes   = interpreter.get_tensor(output_details[0]['index'])[0]
    classes = interpreter.get_tensor(output_details[1]['index'])[0]
    scores  = interpreter.get_tensor(output_details[2]['index'])[0]

    for i in range(len(scores)):
        if scores[i] > 0.5:
            ymin, xmin, ymax, xmax = boxes[i]

            x1 = int(xmin * w)
            y1 = int(ymin * h)
            x2 = int(xmax * w)
            y2 = int(ymax * h)

            label = labels[int(classes[i])]
            detections_global.append((x1, y1, x2, y2, label))

            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            # cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

    if tracking and tracker is not None:
        success, bbox = tracker.update(frame)

        if success:
            tx, ty, tw, th = [int(v) for v in bbox]
            selected_box = (tx, ty, tx + tw, ty + th)

            cv2.rectangle(frame, (tx, ty), (tx + tw, ty + th), (0, 245, 160), 3)
            # cv2.putText(frame, "TRACKING", (tx, ty - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 245, 160), 2)
        else:
            best_iou = 0
            best_box = None
            for box in detections_global:
                x1, y1, x2, y2, label = box
                score = compute_iou(selected_box, (x1, y1, x2, y2))
                if score > best_iou:
                    best_iou = score
                    best_box = (x1, y1, x2, y2)

            if best_iou > 0.1 and best_box:
                bx1, by1, bx2, by2 = best_box
                tracker = cv2.TrackerCSRT_create()
                tracker.init(frame, (bx1, by1, bx2 - bx1, by2 - by1))

    cv2.imshow("Object Detection", frame)   
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break
    elif key == ord('r'):
        tracking = False
        tracker = None
        selected_box = None

cap.release()
cv2.destroyAllWindows()

/home/premesh/vs_code/object_focus/.venv/lib/python3.12/site-packages/tensorflow/lite/python/interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
[ WARN:0@196.109] global cap_v4l.cpp:914 open VIDEOIO(V4L2:/dev/video0): can't open camera by index
[video4linux2,v4l2 @ 0x42b9bc80] ioctl(VIDIOC_G_INPUT): Inappropriate ioctl for device
[ERROR:0@196.110] global obsensor_uvc_stream_channel.cpp:163 getStreamChannelGroup Camera index out of range


In [16]:
import cv2
import numpy as np
from ultralytics import YOLO

model = YOLO('yolo11n-seg.pt')

tracking = False
tracked_id = None
tracked_label = ""
current_results = None
prev_box = None

focus_strength = 0.0
FOCUS_SPEED_UP = 0.06
FOCUS_SPEED_DOWN = 0.08


def on_trackbar(val):
    pass


def mouse_click(event, x, y, flags, param):
    global tracking, tracked_id, tracked_label, current_results, focus_strength

    if event == cv2.EVENT_LBUTTONDOWN and current_results:
        boxes = current_results[0].boxes
        if boxes.id is not None:

            best_idx = None
            best_area = float('inf')

            for i in range(len(boxes)):
                x1, y1, x2, y2 = map(int, boxes.xyxy[i])

                if x1 <= x <= x2 and y1 <= y <= y2:
                    area = (x2 - x1) * (y2 - y1)
                    if area < best_area:
                        best_area = area
                        best_idx = i

            if best_idx is not None:
                tracked_id = int(boxes.id[best_idx])
                tracked_label = model.names[int(boxes.cls[best_idx])]
                tracking = True
                focus_strength = 0.0
                print(f"Now tracking: {tracked_label} (ID: {tracked_id})")


cap = cv2.VideoCapture(0)
cv2.namedWindow("Cinematic Mode")
cv2.setMouseCallback("Cinematic Mode", mouse_click)

cv2.createTrackbar("Blur Level", "Cinematic Mode", 99, 150, on_trackbar)
cv2.createTrackbar("Padding",    "Cinematic Mode", 15,  50,  on_trackbar)


while True:
    ret, frame = cap.read()
    if not ret:
        break

    results = model.track(frame, persist=True, verbose=False, conf=0.15, iou=0.4, retina_masks=True)
    current_results = results

    boxes = results[0].boxes
    masks = results[0].masks

    blur_slider = cv2.getTrackbarPos("Blur Level", "Cinematic Mode")

    if tracking:
        focus_strength = min(1.0, focus_strength + FOCUS_SPEED_UP)
    else:
        focus_strength = max(0.0, focus_strength - FOCUS_SPEED_DOWN)

    dynamic_blur = int(max(1, blur_slider * focus_strength))
    if dynamic_blur % 2 == 0:
        dynamic_blur += 1

    blurred_frame = cv2.GaussianBlur(frame, (dynamic_blur, dynamic_blur), 0)

    output_frame = frame.copy()

    if tracking and boxes.id is not None and masks is not None:

        object_found_this_frame = False

        for i in range(len(boxes)):
            current_id = int(boxes.id[i])

            if current_id == tracked_id:
                object_found_this_frame = True

                mask = masks.data[i].cpu().numpy()
                mask_resized = cv2.resize(mask, (frame.shape[1], frame.shape[0]))

                pad_val = cv2.getTrackbarPos("Padding", "Cinematic Mode")
                if pad_val > 0:
                    kernel = np.ones((pad_val, pad_val), np.uint8)
                    mask_processed = cv2.dilate(mask_resized, kernel, iterations=1)
                else:
                    mask_processed = mask_resized.copy()

                mask_3d = np.repeat(mask_processed[:, :, np.newaxis], 3, axis=2).astype(np.float32)
                feathered_mask = cv2.GaussianBlur(mask_3d, (21, 21), 0)

                output_frame = (feathered_mask * frame + (1 - feathered_mask) * blurred_frame).astype(np.uint8)

                x1, y1, x2, y2 = map(int, boxes.xyxy[i])
                prev_box = (x1, y1, x2, y2)

                cv2.putText(output_frame, tracked_label, (x1, max(30, y1 - 10)),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

                break

        if not object_found_this_frame:
            if prev_box is not None and boxes.id is not None:

                best_dist = float('inf')
                new_target = None
                pcx = (prev_box[0] + prev_box[2]) // 2
                pcy = (prev_box[1] + prev_box[3]) // 2

                for i in range(len(boxes)):
                    x1, y1, x2, y2 = map(int, boxes.xyxy[i])
                    cx = (x1 + x2) // 2
                    cy = (y1 + y2) // 2
                    dist = (cx - pcx) ** 2 + (cy - pcy) ** 2

                    if dist < best_dist:
                        best_dist = dist
                        new_target = i

                if new_target is not None and best_dist < 150**2:
                    tracked_id = int(boxes.id[new_target])
                else:
                    output_frame = blurred_frame
            else:
                output_frame = blurred_frame

    else:
        if boxes.id is not None:
            for i in range(len(boxes)):
                x1, y1, x2, y2 = map(int, boxes.xyxy[i])
                label = model.names[int(boxes.cls[i])]
                cv2.rectangle(output_frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(output_frame, label, (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

    cv2.imshow("Cinematic Mode", output_frame)

    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break
    elif key == ord('r'):
        tracking = False
        tracked_id = None
        tracked_label = ""
        prev_box = None
        focus_strength = 0.0

cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)


Now tracking: person (ID: 3)
Now tracking: person (ID: 1)
Now tracking: person (ID: 21)
Now tracking: person (ID: 1)
Now tracking: remote (ID: 88)
Now tracking: person (ID: 84)
Now tracking: cell phone (ID: 151)
Now tracking: person (ID: 1)
Now tracking: person (ID: 227)
Now tracking: person (ID: 1)
Now tracking: person (ID: 227)
Now tracking: person (ID: 274)
Now tracking: person (ID: 227)
Now tracking: person (ID: 227)
Now tracking: person (ID: 522)
Now tracking: person (ID: 553)
Now tracking: person (ID: 522)


KeyboardInterrupt: 